<h1  align="center"> <b> Music Generator </b></h1>
<h4 align="center">By Sharoon Yaqub</h4>

In [ ]:
#Imports
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from music21 import converter, instrument, note, chord, stream

import os
import tarfile
import urllib.request


In [ ]:
# Downloading Data 

url = 'http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz'
save_path = 'lmd_full.tar.gz'
extract_path = 'lmd_full'

print("Downloading LMD...")
urllib.request.urlretrieve(url, save_path)
print("Download complete!")

print("Extracting LMD...")
with tarfile.open(save_path, 'r:gz') as tar:
    tar.extractall(extract_path)
print("Extraction complete!")

In [ ]:
# Loading Data 
def load_notes(file_path):
    midi = converter.parse(file_path)
    notes = []

    # Traverse all the elements in the MIDI file
    for element in midi.flat:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [ ]:
def prepare_sequences(notes, sequence_length):
    pitch_names = sorted(set(item for item in notes))
    pitch_to_int = dict((pitch, idx) for idx, pitch in enumerate(pitch_names))
    network_input = []
    network_output = []

    # Create sequences of input notes and corresponding target notes
    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([pitch_to_int[pitch] for pitch in sequence_in])
        network_output.append(pitch_to_int[sequence_out])

    num_patterns = len(network_input)
    input_data = np.reshape(network_input, (num_patterns, sequence_length, 1))
    input_data = input_data / float(len(pitch_names))
    output_data = np.array(network_output)

    return input_data, output_data, pitch_names

def generate_music(model, input_data, pitch_names, sequence_length, num_notes):
    start = np.random.randint(0, len(input_data) - 1)
    pattern = list(input_data[start])
    prediction_output = []

    # Generate notes
    for _ in range(num_notes):
        prediction_input = np.reshape(pattern, (1, sequence_length, 1))
        prediction_input = prediction_input / float(len(pitch_names))

        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = pitch_names[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return prediction_output

notes = load_notes('music.mid')

sequence_length = 100
num_notes = 500

input_data, output_data, pitch_names = prepare_sequences(notes, sequence_length)

# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(input_data.shape[1], input_data.shape[2]), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(pitch_names), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Train the model
model.fit(input_data, output_data, epochs=50, batch_size=64)

generated_notes = generate_music(model, input_data, pitch_names, sequence_length, num_notes)

midi_stream = stream.Stream()
for pattern in generated_notes:
    if '.' in pattern:
        notes_in_chord = pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    else:
        new_note = note.Note(int(pattern))
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)

midi_stream.write('midi', fp='generated_music.mid')